<a href="https://colab.research.google.com/github/jisoolee11/AI_study/blob/main/DACON/computer_vision(test1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> DACON 제 2회 컴퓨터 비전 학습 경진대회

In [7]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
!unzip -qq /content/gdrive/My\ Drive/data_2/mnist_data.zip
!unzip -qq /content/gdrive/MyDrive/data_2/test_dirty_mnist.zip
!unzip -qq /content/gdrive/MyDrive/data_2/dirty_mnist.zip

In [9]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

from torchvision import transforms
from torchvision.models import resnet50

In [10]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

In [11]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [12]:
trainset = MnistDataset('/content', '/content/gdrive/MyDrive/data_2/dirty_mnist_answer.csv', transforms_train)
testset = MnistDataset('/content', '/content/gdrive/MyDrive/data_2/sample_submission.csv', transforms_test)

train_loader = DataLoader(trainset, batch_size=32, num_workers=2)
test_loader = DataLoader(testset, batch_size=10, num_workers=1)

In [13]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel().to(device)
print(summary(model,(3, 256, 256)))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,408
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]           4,096
       BatchNorm2d-6           [-1, 64, 64, 64]             128
              ReLU-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
             ReLU-10           [-1, 64, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]          16,384
      BatchNorm2d-12          [-1, 256, 64, 64]             512
           Conv2d-13          [-1, 256, 64, 64]          16,384
      BatchNorm2d-14          [-1, 256

In [14]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()

num_epochs = 5
model.train()

for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f'{epoch}: {loss.item():.5f}, {acc.item():.5f}')

0: 0.70214, 0.54447
0: 0.70570, 0.52043
0: 0.71635, 0.52885
0: 0.70091, 0.53365
0: 0.69968, 0.52764
0: 0.69889, 0.55168
0: 0.69890, 0.54087
0: 0.69700, 0.52404
0: 0.69483, 0.53245
0: 0.69598, 0.53245
0: 0.70930, 0.51683
0: 0.69159, 0.54688
0: 0.69628, 0.53125
0: 0.69599, 0.53726
0: 0.69663, 0.52885
0: 0.69316, 0.54207
0: 0.69584, 0.52644
0: 0.68876, 0.54207
0: 0.69461, 0.52764
0: 0.69209, 0.55168
0: 0.69348, 0.53125
0: 0.69098, 0.54688
0: 0.69012, 0.53005
0: 0.69332, 0.53966
0: 0.69081, 0.54688
0: 0.69268, 0.53726
0: 0.68984, 0.54087
0: 0.68520, 0.55409
0: 0.68426, 0.53486
0: 0.68710, 0.55288
0: 0.69062, 0.53606
0: 0.68105, 0.54688
0: 0.68899, 0.53005
0: 0.68612, 0.52404
0: 0.69211, 0.53125
0: 0.69060, 0.54447
0: 0.69140, 0.52644
0: 0.68627, 0.53726
0: 0.68886, 0.54447
0: 0.68424, 0.55649
0: 0.68396, 0.53245
0: 0.68910, 0.51683
0: 0.68606, 0.53365
0: 0.68611, 0.54928
0: 0.68394, 0.54327
0: 0.68357, 0.54327
0: 0.68353, 0.53606
0: 0.68627, 0.54447
0: 0.68791, 0.53486
0: 0.68545, 0.53966


In [16]:
submit = pd.read_csv('/content/gdrive/MyDrive/data_2/sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()
    
submit.to_csv('submit1.csv', index=False)
from google.colab import files
files.download('submit1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>